In [1]:
import tqdm
import tensorflow as tf
import numpy as np
import tqdm
import pickle
import numpy
import os
import csv
from mittens import Mittens
from sklearn.feature_extraction import _stop_words
from sklearn.feature_extraction.text import CountVectorizer
from scipy import spatial 

In [2]:
%run constants.py
tf.random.set_seed(
    SEED
)
np.random.seed(SEED)

In [3]:
with open(X_TRAIN_INPUT_SAVE_FILE_PRE_VEC, "rb") as f:
        x_train = pickle.load(f)
with open(VOCAB_SAVE_FILE, "rb") as f:
        vocab = pickle.load(f)

In [4]:
def get_embedding_index():
    embeddings_index = {}

    f = open(GLOVE_300D_FILEPATH)
    for line in tqdm.tqdm(f, ncols=100, desc="Loading Glove Embeddings."):
        values = line.split()
        word = values[0]
        coefs = numpy.asarray(values[1:], dtype="float32")
        embeddings_index[word] = coefs
    f.close()

    return embeddings_index

In [5]:
ei = get_embedding_index()

Loading Glove Embeddings.: 400000it [00:29, 13532.05it/s]


In [6]:
train_corpora = x_train[29].str.split()
train_corpora = train_corpora.tolist()
flat_list_train_corpora = [x for xs in train_corpora for x in xs]
flat_list_train_corpora = list(set(flat_list_train_corpora))
flat_list_train_corpora

['saw',
 'greenwood',
 'ux',
 'sued',
 'sewer',
 'pentax',
 'induction',
 'cookbook',
 'hook',
 'sentencing',
 'hygiene',
 'surveyor',
 'quarter',
 'enrichment',
 'believe',
 'feb',
 'banking',
 'shallow',
 'broadcast',
 'grain',
 'slopes',
 'unwanted',
 'adhesion',
 'atmosphere',
 'chimney',
 'angry',
 'running',
 'detection',
 'molecule',
 'l',
 'upset',
 'compromise',
 'vacancy',
 'reap',
 'ppp',
 'phenomenon',
 'detached',
 'elizabeth',
 'tcl',
 'irvine',
 'paypal',
 'requesting',
 'distinctive',
 'suburbs',
 'diecast',
 'script',
 'veterans',
 'diazepam',
 'prism',
 'registration',
 'ribs',
 'products',
 'connie',
 'white',
 'charleston',
 'emotional',
 'slice',
 'ideology',
 'lange',
 'mills',
 'subversion',
 'terminals',
 'sculptures',
 'allah',
 'audits',
 'woke',
 'routinely',
 'bishop',
 'audit',
 'utilizes',
 'rolex',
 'estrogen',
 'scopes',
 'bargain',
 'software',
 'henri',
 'freedoms',
 'pediatrics',
 'vail',
 'ambient',
 'suffer',
 'basically',
 'asm',
 'medical',
 'supp

In [7]:

def glove2dict(glove_filename):
    with open(glove_filename, encoding='utf-8') as f:
        reader = csv.reader(f, delimiter=' ', quoting=csv.QUOTE_NONE)
        embed = {line[0]: np.array(list(map(float, line[1:])))
                for line in reader}
    return embed


pre_glove = glove2dict(GLOVE_300D_FILEPATH)
sw = list(_stop_words.ENGLISH_STOP_WORDS)
train_corpora_for_glove = [token.lower() for token in flat_list_train_corpora if (token.lower() not in sw)]
oov = [token for token in train_corpora_for_glove if token not in pre_glove.keys()]

corp_vocab = list(set(oov))
brown_doc = [' '.join(train_corpora_for_glove)]

cv = CountVectorizer(ngram_range=(1,1), vocabulary=corp_vocab)
X = cv.fit_transform(brown_doc)
Xc = (X.T * X)
Xc.setdiag(0)
coocc_ar = Xc.toarray()

mittens_model = Mittens(n=EMBEDDING_DIM, max_iter=10000)

new_embeddings = mittens_model.fit(
    coocc_ar,
    vocab=corp_vocab,
    initial_embedding_dict= pre_glove)

newglove = dict(zip(corp_vocab, new_embeddings))


c:\Users\aaron\anaconda3\envs\tf-gpu\lib\site-packages\scipy\sparse\_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Iteration 1870: loss: 0.00010127962741535157

In [8]:
new_glove_merged = newglove | ei
len(new_glove_merged)

400145

In [9]:
try:
    os.remove(FINE_TUNED_GLOVE_300D_FILEPATH)
except OSError:
    pass
with open(FINE_TUNED_GLOVE_300D_FILEPATH, "w") as file:
    for k, v in new_glove_merged.items():
        dimensions = ""
        for x in v:
            dimensions += str(x) + " "
        glove_line = k + " " + dimensions
        file.write(glove_line)

In [10]:

def find_closest_embeddings(embedding_index, embedding): 
    return sorted(embedding_index.keys(), key=lambda word: spatial.distance.euclidean(embedding_index[word], embedding))

In [11]:
find_closest_embeddings(new_glove_merged, new_glove_merged['libstdc'])[0:10]

['libstdc',
 'http://www.mediabynumbers.com',
 'str95bb',
 'k977-1',
 'mo95',
 'js94bb',
 'bulletinyyy',
 'bb96',
 'k587-1',
 'bdb94']

In [12]:
#Should fail - as libstdc is NOT in the original glove matrix
find_closest_embeddings(ei, ei['libstdc'])[0:10]

KeyError: 'libstdc'